In [1]:
#| hide
from longreadtools.core import *

# longreadtools

> A working repository for development of tools for the analasis of long read rna sequencing data.

## Install

```sh
pip install git+https://github.com/cobioda/longreadtools.git

```

## How to use

Here we will use the `isomatrix_tools` module to convert our isomatrix txt files in bulk into `anndata` objects using the `multiple_isomatrix_conversion` function . Then, we will utilize our specialized `concatenate_anndata` function to generate a concatenated `anndata` for downstream analysis.


In [2]:
from longreadtools.isomatrix_tools import *

In [3]:
import os
import re

directory = '/data/analysis/data_mcandrew/000-sclr-discovair/'

# Define the regular expression pattern
pattern = re.compile('.*(_BIOP_INT|BIOP_NAS)$')

# Get a list of all files in the directory
all_files = os.listdir(directory)

# Filter the list to include only files that match the pattern
matching_files = [os.path.join(directory, f) for f in all_files if pattern.match(f)]

# Print the list of matching files
print(matching_files)

individual_runs = matching_files
individual_runs = [f'{run}_isomatrix.txt' for run in individual_runs]
individual_runs = [os.path.join(run, f'{os.path.basename(run)}_isomatrix.txt') for run in matching_files]

isomatrice_paths = individual_runs


['/data/analysis/data_mcandrew/000-sclr-discovair/D498_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D492_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D494_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D500_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D494_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D496_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D499_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D493_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D493_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D534_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D490_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D500_BIOP_NAS', '/data/analysis/data_mcandrew/000-sclr-discovair/D495_BIOP_INT', '/data/analysis/data_mcandrew/000-sclr-discovair/D492_BIOP_INT']


In [4]:
converted_isomatrix_paths = multiple_isomatrix_conversion(isomatrice_paths)
andata_concat = concatenate_anndata(converted_isomatrix_paths)

In [ ]:
andata_concat

In [ ]:
andata_concat.var